# 环境与分布偏移

当数据的分布发生变化时，可能会导致模型失效甚至完全失效。换句话说，通过将基于模型的决策引入环境，我们可能会破坏模型。

## 分布偏移类型

假设训练数据是从分布 $p_S(\mathbf{x},y)$ 中采样的，而测试数据则是从分布 $p_T(\mathbf{x},y)$ 中采样。那么，如果完全不知道 $p_S$ 和 $p_T$ 之间的关系，那么不可能为这种偏移进行纠正。反之如果知道数据可能发生变化的一些限制性假设下，则可以进行纠正，保持模型有效性。

### 协变量偏移 （covariate shift）

假设：虽然输入的分布可能改变， 但标签函数（即条件分布 $P(y \mid \mathbf{x})$）没有改变。

### 标签偏移 (label shift)

假设：标签边缘概率 $P(y)$ 可以改变， 但是类别条件分布 $P(\mathbf{x} \mid y)$ 在不同的领域之间保持不变。

例子：根据症状预测疾病类型

### 概念偏移 (concept shift)

假设：标签的定义可能会发生变化。

例子：不同地方的人对于同一个事物的叫法。

## 分布偏移纠正

当进行训练时，实际上是在尝试通过最小化经验风险（empirical risk）来最小化真实风险（true risk）。

经验风险：

$$
\mathop{\mathrm{minimize}}_f \frac{1}{n} \sum_{i=1}^n l(f(\mathbf{x}_i), y_i)
$$

真实风险：

$$
E_{p(\mathbf{x}, y)} [l(f(\mathbf{x}), y)] = \int\int l(f(\mathbf{x}), y) p(\mathbf{x}, y) \;d\mathbf{x}dy
$$

### 协变量偏移纠正

terms:

- 源分布：$q(\mathbf{x})$
- 目标分布：$p(\mathbf{x})$
- 假设：$p(y \mid \mathbf{x}) = q(y \mid \mathbf{x})$

所以有：

$$
\begin{aligned}
\int\int l(f(\mathbf{x}), y) p(y \mid \mathbf{x})p(\mathbf{x}) \;d\mathbf{x}dy =
\int\int l(f(\mathbf{x}), y) q(y \mid \mathbf{x})q(\mathbf{x})\frac{p(\mathbf{x})}{q(\mathbf{x})} \;d\mathbf{x}dy.
\end{aligned}
$$

也就是说，只需要加入目标分布和源分布的概率比作为权重：

$$
\beta_i \stackrel{\mathrm{def}}{=} \frac{p(\mathbf{x}_i)}{q(\mathbf{x}_i)}
$$

在训练时尝试：

$$
\mathop{\mathrm{minimize}}_f \frac{1}{n} \sum_{i=1}^n \beta_i l(f(\mathbf{x}_i), y_i).
$$

而获取两个分布的不同，可以使用 logistic regression：从源分布和目标分布中随机取样，并分别赋予标签 $z$ 为 1（源分布）或者 -1（目标分布）。那么有：

$$
P(z=1 \mid \mathbf{x}) = \frac{p(\mathbf{x})}{p(\mathbf{x})+q(\mathbf{x})} \text{ and hence } \frac{P(z=1 \mid \mathbf{x})}{P(z=-1 \mid \mathbf{x})} = \frac{p(\mathbf{x})}{q(\mathbf{x})}.
$$

又使用 logistic regression：

$$
P(z=1 \mid \mathbf{x})=\frac{1}{1+\exp(-h(\mathbf{x}))}
$$

其中 $h$ 是一个参数化函数。故

$$
\beta_i = \frac{1/(1 + \exp(-h(\mathbf{x}_i)))}{\exp(-h(\mathbf{x}_i))/(1 + \exp(-h(\mathbf{x}_i)))} = \exp(h(\mathbf{x}_i)).
$$

那么，权重 $\beta_i$ 就可以通过训练函数 $h(x)$ 来获得了。问题：需要目标分布(例如，测试分布)中的每个数据样本在训练时出现的概率非零。否则相应的权重将会为 $\infty$。

### 标签偏移纠正

- 源分布：$q$
- 目标分布：$p$
- 假设：$q(y) \neq p(y)$，但 $q(\mathbf{x} \mid y)=p(\mathbf{x} \mid y)$

那么：

$$
\begin{aligned}
\int\int l(f(\mathbf{x}), y) p(\mathbf{x} \mid y)p(y) \;d\mathbf{x}dy =
\int\int l(f(\mathbf{x}), y) q(\mathbf{x} \mid y)q(y)\frac{p(y)}{q(y)} \;d\mathbf{x}dy.
\end{aligned}
$$

这时的权重为：

$$
\beta_i \stackrel{\mathrm{def}}{=} \frac{p(y_i)}{q(y_i)}.
$$

由于 $q(y_i)$ 是可以求得的，那么就只需要求（近似）解 $p(y_i)$。

如果在源分布上有一个非常好的模型，并且目标数据只包含我们以前见过的类别，那么可以通过引入一个**混淆矩阵** $\mathbf{C}$ 来完成对 $p$ 的近似，其中 $\mathbf{C}$ 是一个 $k \times k$ 矩阵，行代表真实标签，列代表模型预测标签：

将所有模型在测试时的预测取平均数， 得到平均模型输出 $\mu(\hat{\mathbf{y}}) \in \mathbb{R}^k$，那么有：

$$
\mathbf{C} p(\mathbf{y}) = \mu(\hat{\mathbf{y}})
$$

又由于模型原本效果就非常好，所以 $\mathbf{C}$ 大概率是可逆的，就可以求得：

$$
p(\mathbf{y}) = \mathbf{C}^{-1} \mu(\hat{\mathbf{y}})
$$

### 概念偏移纠正

难以处理，也难以遇到。可以使用新数据更新现有的网络权重，而不是从头开始训练。

